In [1]:
from transformers import pipeline, set_seed


import matplotlib.pyplot as plt
import datasets
import pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from tqdm import tqdm

from transformers import DataCollatorForSeq2Seq

2024-03-02 00:32:10.675367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 00:32:10.675506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 00:32:10.817722: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data_csv=pd.read_csv("/kaggle/input/malayalam-news/news_90k.csv")

In [3]:
data_csv.describe()

,headline,para
count,89934,89934
unique,89934,89934
top,പ്രശാന്ത് കിഷോറിന് കോണ്‍ഗ്രസില്‍ പരസ്യ പിന്തുണ...,ദില്ലി: പ്രശാന്ത് കിഷോറിനെ കോണ്‍ഗ്രസിലേക്ക് സ്...
freq,1,1


In [4]:
data_csv.nunique()

headline    89934
para        89934
dtype: int64

In [5]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [6]:
model_ckpt="facebook/nllb-200-distilled-600M"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to("cuda")

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [8]:
train_dataset=data_csv[:70000]
val_dataset=data_csv[70000:]

In [9]:
val_dataset.reset_index(drop=True,inplace=True)

In [10]:
train_data=datasets.Dataset.from_pandas(train_dataset)
val_data=datasets.Dataset.from_pandas(val_dataset)

In [11]:
dataset_samsum=datasets.DatasetDict({"train":train_data,"validation":val_data})

In [12]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['headline', 'para'],
        num_rows: 70000
    })
    validation: Dataset({
        features: ['headline', 'para'],
        num_rows: 19934
    })
})

In [13]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['para'] , max_length = 256, truncation = True )


    target_encodings = tokenizer(example_batch['headline'], max_length = 64, truncation = True )

# Returning the Tokenized Features:
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)


  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

In [14]:
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['headline', 'para', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 70000
    })
    validation: Dataset({
        features: ['headline', 'para', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 19934
    })
})

In [15]:


seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='samsum', 
    num_train_epochs=2, 
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01, 
    logging_steps=10,
    evaluation_strategy='steps', 
    eval_steps=500, 
    save_steps=1e6,
    gradient_accumulation_steps=16,
    report_to="none"
)

In [17]:
trainer = Trainer(model=model, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])
trainer.train()

Step,Training Loss,Validation Loss
500,1.964200,2.291166
1000,1.968900,2.190380
1500,1.780100,2.151663
2000,1.835600,2.114233
2500,1.706700,2.086366
3000,1.696500,2.077085
3500,1.869400,2.063992
4000,1.662700,2.020458
4500,1.750200,2.041054
5000,1.619800,2.028684


TrainOutput(global_step=8750, training_loss=1.7209568176269532, metrics={'train_runtime': 28747.0054, 'train_samples_per_second': 4.87, 'train_steps_per_second': 0.304, 'total_flos': 3.899101582270464e+16, 'train_loss': 1.7209568176269532, 'epoch': 2.0})

In [18]:
model.save_pretrained("samsum-model","tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


In [19]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

pipe = pipeline("summarization", model="samsum-model",tokenizer=tokenizer)

In [20]:
for i in range(10):
    sample_text = dataset_samsum["validation"][i]["para"]

    reference = dataset_samsum["validation"][i]["headline"]


    print("Dialogue:")
    print(sample_text)


    print("\nReference Summary:")
    print(reference)


    print("\nModel Summary:")
    print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])
    print("----------------------------------------------------------------------------------------")

Dialogue:
ദില്ലി: സബ്‌സിഡിയില്ലാത്ത പാചകവാതക സിലിണ്ടറിനു വില കുറച്ചു. 14.50 രൂപയാണ് ഇത്തരം സിലിണ്ടറുകള്‍ക്കു കുറവ് വന്നിട്ടുള്ളത്. എന്നാല്‍ സബ്‌സിഡിയുള്ള പാചക വാതക സിലിണ്ടറിനു വില കൂട്ടുകയും ചെയ്തു. 5.57 രൂപയാണ് വര്‍ധിപ്പിച്ചിട്ടുള്ളത്. സബ്‌സിഡിയില്ലാത്ത സിലിണ്ടറിനു നേരത്തേ 737.50 രൂപയായിരുന്നു വില. ഇത് ഇനി 723 രൂപയ്ക്കു ലഭിക്കും. മാര്‍ച്ചില്‍ സബ്‌സിഡിയില്ലാത്ത സിലിണ്ടറുകള്‍ക്കു 86 രൂപ വര്‍ധിപ്പിച്ചിരുന്നു. അതേസമയം, വിമാന ഇന്ധന വിലയില്‍ അഞ്ചു ശതമാനം കുറവ് വരുത്തിയിട്ടുണ്ട്. ഏപ്രില്‍ ഒന്നു മുതല്‍ പുതിയ നിരക്കുകള്‍ പ്രാബല്യത്തില്‍ വന്നു.

Reference Summary:
സബ്‌സിഡിയില്ലാത്ത പാചകവാതക സിലിണ്ടര്‍ വില കുറഞ്ഞു, സബ്‌സിഡിയുള്ളത് വില കൂടും!!

Model Summary:
സബ് സിഡി ഇല്ലാത്ത പാചകവാതക സിലിണ്ടറിന് വില കുറച്ചു
----------------------------------------------------------------------------------------
Dialogue:
പരിധിയില്ലാത്ത ഓഫറുകളുമായി കഴിഞ്ഞ സെപ്തംബറിലാണ് ടെലികോം മേഖലയെ ഒന്നടങ്കം ഞെട്ടിച്ചുകൊണ്ട് ജിയോ രംഗത്ത് എത്തുന്നത്. ഏപ്രില്‍ ഒന്ന് വരെയായിരുന്നു ജിയോയുടെ അണ്‍ലിമിറ്റഡ് ഓഫറുകള്‍ പ്രഖ്യാപിച്ചത്. എ

Your max_length is set to 128, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)


ആര് കെ നഗറില് പാര് ട്ടി ഓഫീസ് ലൈംഗികബന്ധത്തിന് ഉപയോഗിച്ച ദൃശ്യങ്ങള് പുറത്ത്
----------------------------------------------------------------------------------------
Dialogue:
ദില്ലി: ഹിന്ദുവായ അങ്കിതയും മുസ്ലീമായ ഫായിസ് റഹ്മാനും വിവാഹിതരായി. ഇതില്‍ ഇപ്പൊ എന്താ ഇരിയ്ക്കുന്നത് എന്നല്ലെ... കാര്യമുണ്ട്... ഒരു തവണ അല്ല, നാല് തവണയാണ് ഇരുവരും വിവാഹിതരായത്. അതിന് ഒരു കാരണവുമുണ്ട്. ഇതാ അങ്കിതയുടേയും ഫായിസിന്റെ പ്രണയ കഥ.

Reference Summary:
ഫായിസ് അങ്കിതയെ വിവാഹം ചെയ്തത് നാല് തവണ...!!! ഇപ്പോള്‍ സംശയം മാറിയില്ലെ...?

Model Summary:
അങ്കിതയും ഫായിസും വിവാഹിതരായി
----------------------------------------------------------------------------------------
Dialogue:
ദില്ലി: അക്കൗണ്ട് ഉടമകള്‍ക്ക് സീറോ ആന്വല്‍ ഫീ ക്രെഡിറ്റ് കാര്‍ഡുമായി സ്റ്റേറ്റ് ബാങ്ക് ഓഫ് ഇന്ത്യ. 20,000 മുതല്‍ 25,000 രൂപ വരെ അക്കൗണ്ട് ബാലന്‍സുള്ളവര്‍ക്കാണ് കാര്‍ഡ് അനുവദിക്കുക. സിബില്‍ സ്‌കോറോ ക്രെഡിറ്റ് കാര്‍ഡ് ഹിസ്റ്ററിയോ പരിഗണിക്കാതെ കാര്‍ഡ് ഉപയോഗിക്കാനുള്ള സൗകര്യവും ഇതോടെ ലഭിയ്ക്കും. രാജ്യത്ത് ക്രെഡിറ്റ് ബാങ്കിൽ 20000-25000 വരെ ബാലൻസു